# Simple Test 

In [ ]:
G = nx.Graph()
G.add_edge(0, 1)
G.add_edge(0, 2)
G.add_edge(0, 3)
G.add_edge(0, 4)
G.add_edge(0, 5)
G.add_edge(0, 6)
G.add_edge(1, 13)
G.add_edge(1, 15)
G.add_edge(3, 10)
G.add_edge(3, 11)
G.add_edge(3, 12)
G.add_edge(3, 15)
G.add_edge(6, 7)
G.add_edge(6, 8)
G.add_edge(6, 8)
G.add_edge(6, 9)
G.add_edge(7, 16)
G.add_edge(7, 14)
nx.draw(G,nodelist=G.nodes, font_color='white' ,node_size = 900, with_labels=True)
nx.write_edgelist(G, '/home/bhandk/Desktop/MAS/real/test.txt')
plt.title("Test Data")
plt.savefig("test")

# Create Synthetic Dataset

In [11]:
import os
import json
import networkx as nx
import networkx.algorithms.community as nx_comm
import numpy as np
import math
from scipy.sparse.linalg import eigsh

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)
    
def get_from_json(file_path):
    with open(file_path) as json_file:
        data = json.load(json_file)
    return data

def molloy_reed(g):
  all_degree =   np.array(g.degree())[:,1]
  degs = np.delete(all_degree,-1)
  k = degs.mean()
  k2 = np.mean(degs ** 2)
  beta = k2/k
  return beta

def global_feature(g): 
    feature = {}
    M = len(g.edges())
    N = len(g)
    degs =   np.array(g.degree())[:,1]
    min_k = np.min(degs)
    k1 = degs.mean()
    k2 = np.mean(degs** 2)
    div = k2 - k1**2
    A = nx.to_scipy_sparse_array(g, weight='weight',dtype=float,format='csr')
    adj_lams = np.sort(eigsh(A, k=N-1, which='LA', return_eigenvectors=False))
    feature["nodes"] = N
    feature["edges"] = M
    feature["heterogeneity"] = div/k1
    feature["density"] = (2*M)/(N*(N-1))
    feature["resilience"] = molloy_reed(g)
    #power_law_exponent = 1 + N / sum(np.log(degs/min_k))
    feature["modularity"] = nx_comm.modularity(g,nx_comm.label_propagation_communities(g))
    try:
        varepsilons = nx.algorithms.distance_measures.eccentricity(g).values()
        feature["eccentricity"]=  np.mean(list(varepsilons))
        feature["diameter"] = nx.algorithms.distance_measures.diameter(g)
        feature["radius"] = nx.algorithms.distance_measures.radius(g)
    except:
        lcc = max(nx.connected_components(g), key=len)
        subGraph = g.subgraph(lcc)
        varepsilons = nx.algorithms.distance_measures.eccentricity(subGraph).values()
        feature["eccentricity"]=  np.mean(list(varepsilons))
        feature["diameter"] = nx.algorithms.distance_measures.diameter(subGraph)
        feature["radius"] = nx.algorithms.distance_measures.radius(subGraph)
    feature["spectral_radius"]=adj_lams[-1]
    feature["spectral_gap"]=  adj_lams[-1]-adj_lams[-2]
    feature["natural_connectivity"]= np.log2(sum(np.exp(np.real(adj_lams)))/len(adj_lams))
    feature["global_efficiency"] = nx.algorithms.efficiency_measures.global_efficiency(g)
    feature["assortativity"] = nx.algorithms.assortativity.degree_assortativity_coefficient(g)
    #global_properties =np.hstack((nodes, edges,density,resilience,heterogeneity,power_law_exponent, modularity,eccentricity,diameter,radius,spectral_radius,spectral_gap,natural_connectivity,assortativity))
    return feature

def gen_graph(cur_n, g_type):
    m = 3
    average_degree = 2 * m 
    p = average_degree / cur_n
    if g_type == 'erdos_renyi':
        g = nx.erdos_renyi_graph(n=cur_n, p=p)
    elif g_type == 'powerlaw':
        g = nx.powerlaw_cluster_graph(n=cur_n, m=3, p=0.05)
    elif g_type == 'small-world':
        g = nx.connected_watts_strogatz_graph(n=cur_n, k=4, p=p)
    elif g_type == 'barabasi_albert':
        g = nx.barabasi_albert_graph(n=cur_n, m=m)
    lcc = max(nx.connected_components(g), key=len)
    Subgraph = g.subgraph(lcc)
    return Subgraph


def get_graph(graph_type,feature):
    noderange = [30, 50, 100, 250, 500, 750]
    properties = []
    files= []
    for graphtype in graph_type:
        print(graphtype, end=", ")
        for nodesize in noderange:
            G= gen_graph(nodesize, graphtype)
            filename = graphtype+"_"+str(nodesize)
            files.append(filename)
            #plt.figure(figsize=(15, 20), dpi=120)
            #nx.draw(G,nodelist=G.nodes, font_color='white' ,node_size = 900, with_labels=True)
            nx.write_edgelist(G, '../Dataset/SyntheticGraph/'+filename+'.txt',data=False)
            feature[filename] = global_feature(G)
    print(files)
    return feature
feature = get_from_json("../Dataset/features.json")
feature = get_graph(['barabasi_albert','small-world','erdos_renyi'], feature)
with open("../Dataset/features.json", "w") as outfile:
    json.dump(feature, outfile, cls=NpEncoder)

barabasi_albert, small-world, erdos_renyi, ['barabasi_albert_30', 'barabasi_albert_50', 'barabasi_albert_100', 'barabasi_albert_250', 'barabasi_albert_500', 'barabasi_albert_750', 'small-world_30', 'small-world_50', 'small-world_100', 'small-world_250', 'small-world_500', 'small-world_750', 'erdos_renyi_30', 'erdos_renyi_50', 'erdos_renyi_100', 'erdos_renyi_250', 'erdos_renyi_500', 'erdos_renyi_750']
{'moreno_crime_projected': {'nodes': 754, 'edges': 2127, 'heterogeneity': 5.480798225168634, 'density': 0.007492576114639585, 'resilience': 11.125088173054314, 'modularity': 0.8006332268597997, 'eccentricity': 12.302387267904509, 'diameter': 16, 'radius': 8, 'spectral_radius': 18.06580694174004, 'spectral_gap': 4.0010810887095705, 'natural_connectivity': 16.558278000253285, 'global_efficiency': 0.17895793744673452, 'assortativity': 0.14183676257165753}, 'corruption': {'nodes': 309, 'edges': 3281, 'heterogeneity': 13.689081689318414, 'density': 0.06894885050224855, 'resilience': 34.93049839

'with open("../Dataset/features.json", "w") as outfile:\n    json.dump(feature, outfile, cls=NpEncoder)'

In [13]:
def get_graph(graph_type,feature):
    noderange = [50,750]
    properties = []
    files= []
    for graphtype in graph_type:
        print(graphtype, end=", ")
        for nodesize in noderange:
            G= gen_graph(nodesize, graphtype)
            filename = graphtype+"_"+str(nodesize)
            files.append(filename)
            #plt.figure(figsize=(15, 20), dpi=120)
            #nx.draw(G,nodelist=G.nodes, font_color='white' ,node_size = 900, with_labels=True)
            nx.write_edgelist(G, '../Dataset/SyntheticGraph/'+filename+'.txt',data=False)
            feature[filename] = global_feature(G)
    print(files)
    return feature
feature = get_from_json("../Dataset/features.json")
feature = get_graph(['erdos_renyi'], feature)
with open("../Dataset/features.json", "w") as outfile:
    json.dump(feature, outfile, cls=NpEncoder)

erdos_renyi, ['erdos_renyi_50', 'erdos_renyi_750']


# Create Heterogeneous CV graph

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

def molloy_reed(g):
  all_degree =   np.array(g.degree())[:,1]
  degs = np.delete(all_degree,-1)
  k = degs.mean()
  k2 = np.mean(degs ** 2)
  if k ==0:
    beta = 0
  else:
    beta = k2/k
  return beta
def global_feature(g): 
    subGraph = g#g.subgraph(np.arange(len(g)-1)) #for supernode
    M = len(subGraph.edges())
    N = len(subGraph)
    degs =   np.array(subGraph.degree())[:,1]
    k1 = degs.mean()
    k2 = np.mean(degs** 2)
    div = k2 - k1**2
    nodes = N
    edges = M
    heterogeneity = div/k1
    density = (2*M)/(N*(N-1))
    resilience = molloy_reed(g)
    global_properties =np.hstack((nodes, edges,density,resilience,heterogeneity)).astype(float)
    return global_properties

def create_heterogeneous_graph(deg,i,num):
    #create a graph with degrees following a power law distribution
    G = nx.expected_degree_graph(deg, selfloops=False)
    G = G.subgraph(max(nx.connected_components(G), key=len))
    sequence = [val for (node, val) in G.degree()]
    #sequence = nx.random_powerlaw_tree_sequence(100,i, tries=5000)
    G_conf = nx.configuration_model(sequence)   
    G_conf.remove_edges_from(nx.selfloop_edges(G_conf))
    G_conf = G_conf.subgraph(max(nx.connected_components(G_conf), key=len))
    feature = global_feature(G_conf)
    cond = True
    if feature[-1] >=20:
        nx.draw(G_conf,nodelist=G_conf.nodes, font_color='white' ,node_size = 50, with_labels=False)
        file_name =  "../Cross_Validation/heterogeneous_"+str(num)
        nx.write_edgelist(G_conf,file_name+str(".txt"))
        plt.savefig(file_name)
        plt.show()
        degree_freq = nx.degree_histogram(G_conf)
        degrees = range(len(degree_freq))
        plt.loglog(degrees, degree_freq,'go-') 
        plt.xlabel('Degree')
        plt.ylabel('Frequency')
        plt.title("Degree Distribution")
        file_name =  "../Cross_Validation/heterogeneous_degree_"
        plt.savefig(file_name+str(num))
        plt.show()
        cond = False
        print("heterogeneous_"+str(num),feature)
    return cond

#for i in np.arange(2.0,3.2,0.2):
cond = True
num = 0 
while cond:
    i = 2.0
    deg = nx.utils.powerlaw_sequence(100, i) #100 nodes, power-law exponent 2.5
    #deg = nx.random_powerlaw_tree_sequence(100, tries=5000)
    cond = create_heterogeneous_graph(deg,i,num)
    if not(cond):
        num += 1
        cond = True
        if num > 10 :
            cond = False


# Create Homogeneous CV graph

In [ ]:
import os

import networkx as nx
import networkx.algorithms.community as nx_comm
import numpy as np
import math
from scipy.sparse.linalg import eigsh

def molloy_reed(g):
  all_degree =   np.array(g.degree())[:,1]
  degs = np.delete(all_degree,-1)
  k = degs.mean()
  k2 = np.mean(degs ** 2)
  beta = k2/k
  return beta 

def get_power_law_exponent(self):
    alpha = 1 + self.N / sum(np.log(self.degs/mink))
    return alpha

    
def global_feature(g): 
    subGraph = g#g.subgraph(np.arange(len(g)-1)) #for supernode
    M = len(subGraph.edges())
    N = len(subGraph)
    degs =   np.array(subGraph.degree())[:,1]
    min_k = np.min(degs)
    
    k1 = degs.mean()
    k2 = np.mean(degs** 2)
    div = k2 - k1**2
    A = nx.to_scipy_sparse_matrix(g, weight='weight',dtype=np.float,format='csr')
    adj_lams = np.sort(eigsh(A, k=N-1, which='LA', return_eigenvectors=False))
    nodes = N
    edges = M
    heterogeneity = div/k1
    density = (2*M)/(N*(N-1))
    resilience = molloy_reed(g)
    power_law_exponent = 1 + N / sum(np.log(degs/min_k))
    modularity = nx_comm.modularity(g,nx_comm.label_propagation_communities(g))
    varepsilons = nx.algorithms.distance_measures.eccentricity(g).values()
    eccentricity=  np.mean(list(varepsilons))
    diameter = nx.algorithms.distance_measures.diameter(g)
    radius = nx.algorithms.distance_measures.radius(g)
    spectral_radius=adj_lams[-1]
    spectral_gap=  adj_lams[-1]-adj_lams[-2]
    natural_connectivity= np.log2(sum(np.exp(np.real(adj_lams)))/len(adj_lams))
    global_efficiency = nx.algorithms.efficiency_measures.global_efficiency(g)
    assortativity = nx.algorithms.assortativity.degree_assortativity_coefficient(g)
    global_properties =np.hstack((nodes, edges,density,resilience,heterogeneity,power_law_exponent, modularity,eccentricity,diameter,radius,spectral_radius,spectral_gap,natural_connectivity))
    return global_properties

def create_homogeneous_graph(num):
    #create a graph with degrees following a power law distribution
    G_conf = nx.erdos_renyi_graph(n=100, p=0.1)
    feature = global_feature(G_conf)
    cond = True
    if feature[-1] <=1:
        nx.draw(G_conf,nodelist=G_conf.nodes, font_color='white' ,node_size = 10, with_labels=False)
        file_name =  "../Cross_Validation/homogeneous_"+str(num)
        nx.write_edgelist(G_conf,file_name+str(".txt"))
        plt.savefig(file_name)
        plt.show()
        degree_freq = nx.degree_histogram(G_conf)
        degrees = range(len(degree_freq))
        plt.loglog(degrees, degree_freq,'go-') 
        plt.xlabel('Degree')
        plt.ylabel('Frequency')
        plt.title("Degree Distribution")
        file_name_deg =  "../Cross_Validation/homogeneous_degree_"
        plt.savefig(file_name_deg+str(num))
        plt.show()
        cond = False
        print("homogeneous_"+str(num),feature)
    return cond

#for i in np.arange(2.0,3.2,0.2):
cond = True
num = 0 
while cond:
    cond = create_homogeneous_graph(num)
    if not(cond):
        num += 1
        cond = True
        if num > 10 :
            cond = False


## Generate Synthetic Graph: GNNExplainer

In [9]:
%matplotlib inline
"""gengraph.py
   Generating and manipulaton the synthetic graphs needed for the paper's experiments.
"""

import os

from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
import matplotlib.colors as colors
from scipy.sparse.linalg import eigsh
# Set matplotlib backend to file writing
#plt.switch_backend("agg")

import networkx as nx
import networkx.algorithms.community as nx_comm
import numpy as np

"""synthetic_structsim.py
    Utilities for generating certain graph shapes.
"""
import math

import networkx as nx
import numpy as np

""" featgen.py
Node feature generators.
"""
import abc


class FeatureGen(metaclass=abc.ABCMeta):
    """Feature Generator base class."""
    @abc.abstractmethod
    def gen_node_features(self, G):
        pass


class ConstFeatureGen(FeatureGen):
    """Constant Feature class."""
    def __init__(self, val):
        self.val = val

    def gen_node_features(self, G):
        feat_dict = {i:{'feat': np.array(self.val, dtype=np.float32)} for i in G.nodes()}
        print ('feat_dict[0]["feat"]:', feat_dict[0]['feat'].dtype)
        nx.set_node_attributes(G, feat_dict)
        print ('G.nodes[0]["feat"]:', G.nodes[0]['feat'].dtype)


class GaussianFeatureGen(FeatureGen):
    """Gaussian Feature class."""
    def __init__(self, mu, sigma):
        self.mu = mu
        if sigma.ndim < 2:
            self.sigma = np.diag(sigma)
        else:
            self.sigma = sigma

    def gen_node_features(self, G):
        feat = np.random.multivariate_normal(self.mu, self.sigma, G.number_of_nodes())
        feat_dict = {
                i: {"feat": feat[i]} for i in range(feat.shape[0])
            }
        nx.set_node_attributes(G, feat_dict)


class GridFeatureGen(FeatureGen):
    """Grid Feature class."""
    def __init__(self, mu, sigma, com_choices):
        self.mu = mu                    # Mean
        self.sigma = sigma              # Variance
        self.com_choices = com_choices  # List of possible community labels

    def gen_node_features(self, G):
        # Generate community assignment
        community_dict = {
            n: self.com_choices[0] if G.degree(n) < 4 else self.com_choices[1]
            for n in G.nodes()
        }

        # Generate random variable
        s = np.random.normal(self.mu, self.sigma, G.number_of_nodes())

        # Generate features
        feat_dict = {
            n: {"feat": np.asarray([community_dict[n], s[i]])}
            for i, n in enumerate(G.nodes())
        }

        nx.set_node_attributes(G, feat_dict)
        return community_dict
    
    
# Following GraphWave's representation of structural similarity


def clique(start, nb_nodes, nb_to_remove=0, role_start=0):
    """ Defines a clique (complete graph on nb_nodes nodes,
    with nb_to_remove  edges that will have to be removed),
    index of nodes starting at start
    and role_ids at role_start
    INPUT:
    -------------
    start       :    starting index for the shape
    nb_nodes    :    int correspondingraph to the nb of nodes in the clique
    role_start  :    starting index for the roles
    nb_to_remove:    int-- numb of edges to remove (unif at RDM)
    OUTPUT:
    -------------
    graph       :    a house shape graph, with ids beginning at start
    roles       :    list of the roles of the nodes (indexed starting at
                     role_start)
    """
    a = np.ones((nb_nodes, nb_nodes))
    np.fill_diagonal(a, 0)
    graph = nx.from_numpy_matrix(a)
    edge_list = graph.edges().keys()
    roles = [role_start] * nb_nodes
    if nb_to_remove > 0:
        lst = np.random.choice(len(edge_list), nb_to_remove, replace=False)
        print(edge_list, lst)
        to_delete = [edge_list[e] for e in lst]
        graph.remove_edges_from(to_delete)
        for e in lst:
            print(edge_list[e][0])
            print(len(roles))
            roles[edge_list[e][0]] += 1
            roles[edge_list[e][1]] += 1
    mapping_graph = {k: (k + start) for k in range(nb_nodes)}
    graph = nx.relabel_nodes(graph, mapping_graph)
    return graph, roles


def cycle(start, len_cycle, role_start=0):
    """Builds a cycle graph, with index of nodes starting at start
    and role_ids at role_start
    INPUT:
    -------------
    start       :    starting index for the shape
    role_start  :    starting index for the roles
    OUTPUT:
    -------------
    graph       :    a house shape graph, with ids beginning at start
    roles       :    list of the roles of the nodes (indexed starting at
                     role_start)
    """
    graph = nx.Graph()
    graph.add_nodes_from(range(start, start + len_cycle))
    for i in range(len_cycle - 1):
        graph.add_edges_from([(start + i, start + i + 1)])
    graph.add_edges_from([(start + len_cycle - 1, start)])
    roles = [role_start] * len_cycle
    return graph, roles


def diamond(start, role_start=0):
    """Builds a diamond graph, with index of nodes starting at start
    and role_ids at role_start
    INPUT:
    -------------
    start       :    starting index for the shape
    role_start  :    starting index for the roles
    OUTPUT:
    -------------
    graph       :    a house shape graph, with ids beginning at start
    roles       :    list of the roles of the nodes (indexed starting at
                     role_start)
    """
    graph = nx.Graph()
    graph.add_nodes_from(range(start, start + 6))
    graph.add_edges_from(
        [
            (start, start + 1),
            (start + 1, start + 2),
            (start + 2, start + 3),
            (start + 3, start),
        ]
    )
    graph.add_edges_from(
        [
            (start + 4, start),
            (start + 4, start + 1),
            (start + 4, start + 2),
            (start + 4, start + 3),
        ]
    )
    graph.add_edges_from(
        [
            (start + 5, start),
            (start + 5, start + 1),
            (start + 5, start + 2),
            (start + 5, start + 3),
        ]
    )
    roles = [role_start] * 6
    return graph, roles


def tree(start, height, r=2, role_start=0):
    """Builds a balanced r-tree of height h
    INPUT:
    -------------
    start       :    starting index for the shape
    height      :    int height of the tree 
    r           :    int number of branches per node 
    role_start  :    starting index for the roles
    OUTPUT:
    -------------
    graph       :    a tree shape graph, with ids beginning at start
    roles       :    list of the roles of the nodes (indexed starting at role_start)
    """
    graph = nx.balanced_tree(r, height)
    roles = [0] * graph.number_of_nodes()
    return graph, roles


def fan(start, nb_branches, role_start=0):
    """Builds a fan-like graph, with index of nodes starting at start
    and role_ids at role_start
    INPUT:
    -------------
    nb_branches :    int correspondingraph to the nb of fan branches
    start       :    starting index for the shape
    role_start  :    starting index for the roles
    OUTPUT:
    -------------
    graph       :    a house shape graph, with ids beginning at start
    roles       :    list of the roles of the nodes (indexed starting at
                     role_start)
    """
    graph, roles = star(start, nb_branches, role_start=role_start)
    for k in range(1, nb_branches - 1):
        roles[k] += 1
        roles[k + 1] += 1
        graph.add_edges_from([(start + k, start + k + 1)])
    return graph, roles


def ba(start, width, role_start=0, m=5):
    """Builds a BA preferential attachment graph, with index of nodes starting at start
    and role_ids at role_start
    INPUT:
    -------------
    start       :    starting index for the shape
    width       :    int size of the graph
    role_start  :    starting index for the roles
    OUTPUT:
    -------------
    graph       :    a house shape graph, with ids beginning at start
    roles       :    list of the roles of the nodes (indexed starting at
                     role_start)
    """
    graph = nx.barabasi_albert_graph(width, m)
    graph.add_nodes_from(range(start, start + width))
    nids = sorted(graph)
    mapping = {nid: start + i for i, nid in enumerate(nids)}
    graph = nx.relabel_nodes(graph, mapping)
    roles = [role_start for i in range(width)]
    return graph, roles


def house(start, role_start=0):
    """Builds a house-like  graph, with index of nodes starting at start
    and role_ids at role_start
    INPUT:
    -------------
    start       :    starting index for the shape
    role_start  :    starting index for the roles
    OUTPUT:
    -------------
    graph       :    a house shape graph, with ids beginning at start
    roles       :    list of the roles of the nodes (indexed starting at
                     role_start)
    """
    graph = nx.Graph()
    graph.add_nodes_from(range(start, start + 5))
    graph.add_edges_from(
        [
            (start, start + 1),
            (start + 1, start + 2),
            (start + 2, start + 3),
            (start + 3, start),
        ]
    )
    # graph.add_edges_from([(start, start + 2), (start + 1, start + 3)])
    graph.add_edges_from([(start + 4, start), (start + 4, start + 1)])
    roles = [role_start, role_start, role_start + 1, role_start + 1, role_start + 2]
    return graph, roles


def grid(start, dim=2, role_start=0):
    """ Builds a 2by2 grid
    """
    grid_G = nx.grid_graph([dim, dim])
    grid_G = nx.convert_node_labels_to_integers(grid_G, first_label=start)
    roles = [role_start for i in grid_G.nodes()]
    return grid_G, roles


def star(start, nb_branches, role_start=0):
    """Builds a star graph, with index of nodes starting at start
    and role_ids at role_start
    INPUT:
    -------------
    nb_branches :    int correspondingraph to the nb of star branches
    start       :    starting index for the shape
    role_start  :    starting index for the roles
    OUTPUT:
    -------------
    graph       :    a house shape graph, with ids beginning at start
    roles       :    list of the roles of the nodes (indexed starting at
                     role_start)
    """
    graph = nx.Graph()
    graph.add_nodes_from(range(start, start + nb_branches + 1))
    for k in range(1, nb_branches + 1):
        graph.add_edges_from([(start, start + k)])
    roles = [role_start + 1] * (nb_branches + 1)
    roles[0] = role_start
    return graph, roles


def path(start, width, role_start=0):
    """Builds a path graph, with index of nodes starting at start
    and role_ids at role_start
    INPUT:
    -------------
    start       :    starting index for the shape
    width       :    int length of the path
    role_start  :    starting index for the roles
    OUTPUT:
    -------------
    graph       :    a house shape graph, with ids beginning at start
    roles       :    list of the roles of the nodes (indexed starting at
                     role_start)
    """
    graph = nx.Graph()
    graph.add_nodes_from(range(start, start + width))
    for i in range(width - 1):
        graph.add_edges_from([(start + i, start + i + 1)])
    roles = [role_start] * width
    roles[0] = role_start + 1
    roles[-1] = role_start + 1
    return graph, roles


def build_graph(
    width_basis,
    basis_type,
    list_shapes,
    start=0,
    rdm_basis_plugins=False,
    add_random_edges=0,
    m=5,
):
    """This function creates a basis (scale-free, path, or cycle)
    and attaches elements of the type in the list randomly along the basis.
    Possibility to add random edges afterwards.
    INPUT:
    --------------------------------------------------------------------------------------
    width_basis      :      width (in terms of number of nodes) of the basis
    basis_type       :      (torus, string, or cycle)
    shapes           :      list of shape list (1st arg: type of shape,
                            next args:args for building the shape,
                            except for the start)
    start            :      initial nb for the first node
    rdm_basis_plugins:      boolean. Should the shapes be randomly placed
                            along the basis (True) or regularly (False)?
    add_random_edges :      nb of edges to randomly add on the structure
    m                :      number of edges to attach to existing node (for BA graph)
    OUTPUT:
    --------------------------------------------------------------------------------------
    basis            :      a nx graph with the particular shape
    role_ids         :      labels for each role
    plugins          :      node ids with the attached shapes
    """
    if basis_type == "ba":
        
        basis, role_id = eval(basis_type)(start, width_basis, m=m)
    else:
        basis, role_id = eval(basis_type)(start, width_basis)
    n_basis, n_shapes = nx.number_of_nodes(basis), len(list_shapes)
    start += n_basis  # indicator of the id of the next node

    # Sample (with replacement) where to attach the new motifs
    if rdm_basis_plugins is True:
        plugins = np.random.choice(n_basis, n_shapes, replace=False)
    else:
        spacing = math.floor(n_basis / n_shapes)
        plugins = [int(k * spacing) for k in range(n_shapes)]
    seen_shapes = {"basis": [0, n_basis]}

    for shape_id, shape in enumerate(list_shapes):
        shape_type = shape[0]
        args = [start]
        if len(shape) > 1:
            args += shape[1:]
        args += [0]
        graph_s, roles_graph_s = eval(shape_type)(*args)
        n_s = nx.number_of_nodes(graph_s)
        try:
            col_start = seen_shapes[shape_type][0]
        except:
            col_start = np.max(role_id) + 1
            seen_shapes[shape_type] = [col_start, n_s]
        # Attach the shape to the basis
        basis.add_nodes_from(graph_s.nodes())
        basis.add_edges_from(graph_s.edges())
        basis.add_edges_from([(start, plugins[shape_id])])
        if shape_type == "cycle":
            if np.random.random() > 0.5:
                a = np.random.randint(1, 4)
                b = np.random.randint(1, 4)
                basis.add_edges_from([(a + start, b + plugins[shape_id])])
        temp_labels = [r + col_start for r in roles_graph_s]
        # temp_labels[0] += 100 * seen_shapes[shape_type][0]
        role_id += temp_labels
        start += n_s

    if add_random_edges > 0:
        # add random edges between nodes:
        for p in range(add_random_edges):
            src, dest = np.random.choice(nx.number_of_nodes(basis), 2, replace=False)
            print(src, dest)
            basis.add_edges_from([(src, dest)])

    return basis, role_id, plugins

####################################
#
# Experiment utilities
#
####################################
def perturb(graph_list, p):
    """ Perturb the list of (sparse) graphs by adding/removing edges.
    Args:
        p: proportion of added edges based on current number of edges.
    Returns:
        A list of graphs that are perturbed from the original graphs.
    """
    perturbed_graph_list = []
    for G_original in graph_list:
        G = G_original.copy()
        edge_count = int(G.number_of_edges() * p)
        # randomly add the edges between a pair of nodes without an edge.
        for _ in range(edge_count):
            while True:
                u = np.random.randint(0, G.number_of_nodes())
                v = np.random.randint(0, G.number_of_nodes())
                if (not G.has_edge(u, v)) and (u != v):
                    break
            G.add_edge(u, v)
        perturbed_graph_list.append(G)
    return perturbed_graph_list


def join_graph(G1, G2, n_pert_edges):
    """ Join two graphs along matching nodes, then perturb the resulting graph.
    Args:
        G1, G2: Networkx graphs to be joined.
        n_pert_edges: number of perturbed edges.
    Returns:
        A new graph, result of merging and perturbing G1 and G2.
    """
    assert n_pert_edges > 0
    F = nx.compose(G1, G2)
    edge_cnt = 0
    while edge_cnt < n_pert_edges:
        node_1 = np.random.choice(G1.nodes())
        node_2 = np.random.choice(G2.nodes())
        F.add_edge(node_1, node_2)
        edge_cnt += 1
    return F


def preprocess_input_graph(G, labels, normalize_adj=False):
    """ Load an existing graph to be converted for the experiments.
    Args:
        G: Networkx graph to be loaded.
        labels: Associated node labels.
        normalize_adj: Should the method return a normalized adjacency matrix.
    Returns:
        A dictionary containing adjacency, node features and labels
    """
    adj = np.array(nx.to_numpy_matrix(G))
    if normalize_adj:
        sqrt_deg = np.diag(1.0 / np.sqrt(np.sum(adj, axis=0, dtype=float).squeeze()))
        adj = np.matmul(np.matmul(sqrt_deg, adj), sqrt_deg)

    existing_node = list(G.nodes)[-1]
    feat_dim = G.nodes[existing_node]["feat"].shape[0]
    f = np.zeros((G.number_of_nodes(), feat_dim), dtype=float)
    for i, u in enumerate(G.nodes()):
        f[i, :] = G.nodes[u]["feat"]

    # add batch dim
    adj = np.expand_dims(adj, axis=0)
    f = np.expand_dims(f, axis=0)
    labels = np.expand_dims(labels, axis=0)
    return {"adj": adj, "feat": f, "labels": labels}


####################################
#
# Generating synthetic graphs
#
###################################
def bahouse(nb_shapes=100, width_basis=60, feature_generator=None, m=5):
    """ Synthetic Graph #1:
    Start with Barabasi-Albert graph and attach house-shaped subgraphs.
    Args:
        nb_shapes         :  The number of shapes (here 'houses') that should be added to the base graph.
        width_basis       :  The width of the basis graph (here 'Barabasi-Albert' random graph).
        feature_generator :  A `FeatureGenerator` for node features. If `None`, add constant features to nodes.
        m                 :  number of edges to attach to existing node (for BA graph)
    Returns:
        G                 :  A networkx graph
        role_id           :  A list with length equal to number of nodes in the entire graph (basis
                          :  + shapes). role_id[i] is the ID of the role of node i. It is the label.
        name              :  A graph identifier
    """
    basis_type = "ba"
    list_shapes = [["house"]] * nb_shapes

    plt.figure(figsize=(8, 6), dpi=300)

    G, role_id, _ = build_graph(
        width_basis, basis_type, list_shapes, start=0, m=5
    )
    G = perturb([G], 0.01)[0]

    if feature_generator is None:
        feature_generator = ConstFeatureGen(1)
    feature_generator.gen_node_features(G)

    name = basis_type + "_" + str(width_basis) + "_" + str(nb_shapes) + "_house"
    return G, role_id, name

def bafan(nb_shapes=100, width_basis=60, feature_generator=None, m=5):
    """ Synthetic Graph #1:
    Start with Barabasi-Albert graph and attach house-shaped subgraphs.
    Args:
        nb_shapes         :  The number of shapes (here 'houses') that should be added to the base graph.
        width_basis       :  The width of the basis graph (here 'Barabasi-Albert' random graph).
        feature_generator :  A `FeatureGenerator` for node features. If `None`, add constant features to nodes.
        m                 :  number of edges to attach to existing node (for BA graph)
    Returns:
        G                 :  A networkx graph
        role_id           :  A list with length equal to number of nodes in the entire graph (basis
                          :  + shapes). role_id[i] is the ID of the role of node i. It is the label.
        name              :  A graph identifier
    """
    basis_type = "ba"
    list_shapes = [["fan"]] * nb_shapes

    plt.figure(figsize=(8, 6), dpi=300)

    G, role_id, _ = build_graph(
        width_basis, basis_type, list_shapes, start=0, m=5
    )
    G = perturb([G], 0.01)[0]

    if feature_generator is None:
        feature_generator = ConstFeatureGen(1)
    feature_generator.gen_node_features(G)

    name = basis_type + "_" + str(width_basis) + "_" + str(nb_shapes) + "_fan"
    return G, role_id, name

def baclique(nb_shapes=100, width_basis=60, feature_generator=None, m=5):
    """ Synthetic Graph #1:
    Start with Barabasi-Albert graph and attach house-shaped subgraphs.
    Args:
        nb_shapes         :  The number of shapes (here 'houses') that should be added to the base graph.
        width_basis       :  The width of the basis graph (here 'Barabasi-Albert' random graph).
        feature_generator :  A `FeatureGenerator` for node features. If `None`, add constant features to nodes.
        m                 :  number of edges to attach to existing node (for BA graph)
    Returns:
        G                 :  A networkx graph
        role_id           :  A list with length equal to number of nodes in the entire graph (basis
                          :  + shapes). role_id[i] is the ID of the role of node i. It is the label.
        name              :  A graph identifier
    """
    basis_type = "ba"
    list_shapes = [["clique"]] * nb_shapes

    plt.figure(figsize=(8, 6), dpi=300)

    G, role_id, _ = build_graph(
        width_basis, basis_type, list_shapes, start=0, m=5
    )
    G = perturb([G], 0.01)[0]

    if feature_generator is None:
        feature_generator = ConstFeatureGen(1)
    feature_generator.gen_node_features(G)

    name = basis_type + "_" + str(width_basis) + "_" + str(nb_shapes) + "_clique"
    return G, role_id, name

def bastar(nb_shapes=100, width_basis=60, feature_generator=None, m=5):
    """ Synthetic Graph #1:
    Start with Barabasi-Albert graph and attach house-shaped subgraphs.
    Args:
        nb_shapes         :  The number of shapes (here 'houses') that should be added to the base graph.
        width_basis       :  The width of the basis graph (here 'Barabasi-Albert' random graph).
        feature_generator :  A `FeatureGenerator` for node features. If `None`, add constant features to nodes.
        m                 :  number of edges to attach to existing node (for BA graph)
    Returns:
        G                 :  A networkx graph
        role_id           :  A list with length equal to number of nodes in the entire graph (basis
                          :  + shapes). role_id[i] is the ID of the role of node i. It is the label.
        name              :  A graph identifier
    """
    basis_type = "ba"
    list_shapes = [["star"]] * nb_shapes

    plt.figure(figsize=(8, 6), dpi=300)

    G, role_id, _ = build_graph(
        width_basis, basis_type, list_shapes, start=0, m=5
    )
    G = perturb([G], 0.01)[0]

    if feature_generator is None:
        feature_generator = ConstFeatureGen(1)
    feature_generator.gen_node_features(G)

    name = basis_type + "_" + str(width_basis) + "_" + str(nb_shapes) + "_star"
    return G, role_id, name

def badiamond(nb_shapes=100, width_basis=60, feature_generator=None, m=5):
    """ Synthetic Graph #1:
    Start with Barabasi-Albert graph and attach house-shaped subgraphs.
    Args:
        nb_shapes         :  The number of shapes (here 'houses') that should be added to the base graph.
        width_basis       :  The width of the basis graph (here 'Barabasi-Albert' random graph).
        feature_generator :  A `FeatureGenerator` for node features. If `None`, add constant features to nodes.
        m                 :  number of edges to attach to existing node (for BA graph)
    Returns:
        G                 :  A networkx graph
        role_id           :  A list with length equal to number of nodes in the entire graph (basis
                          :  + shapes). role_id[i] is the ID of the role of node i. It is the label.
        name              :  A graph identifier
    """
    basis_type = "ba"
    list_shapes = [["diamond"]] * nb_shapes

    plt.figure(figsize=(8, 6), dpi=300)

    G, role_id, _ = build_graph(
        width_basis, basis_type, list_shapes, start=0, m=5
    )
    G = perturb([G], 0.01)[0]

    if feature_generator is None:
        feature_generator = ConstFeatureGen(1)
    feature_generator.gen_node_features(G)

    name = basis_type + "_" + str(width_basis) + "_" + str(nb_shapes) + "_diamond"
    return G, role_id, name


def bacycle(nb_shapes=100, width_basis=60, feature_generator=None, m=5):
    """ Synthetic Graph #1:
    Start with Barabasi-Albert graph and attach house-shaped subgraphs.
    Args:
        nb_shapes         :  The number of shapes (here 'houses') that should be added to the base graph.
        width_basis       :  The width of the basis graph (here 'Barabasi-Albert' random graph).
        feature_generator :  A `FeatureGenerator` for node features. If `None`, add constant features to nodes.
        m                 :  number of edges to attach to existing node (for BA graph)
    Returns:
        G                 :  A networkx graph
        role_id           :  A list with length equal to number of nodes in the entire graph (basis
                          :  + shapes). role_id[i] is the ID of the role of node i. It is the label.
        name              :  A graph identifier
    """
    basis_type = "ba"
    list_shapes = [["cycle"]] * nb_shapes

    plt.figure(figsize=(8, 6), dpi=300)

    G, role_id, _ = build_graph(
        width_basis, basis_type, list_shapes, start=0, m=5
    )
    G = perturb([G], 0.01)[0]

    if feature_generator is None:
        feature_generator = ConstFeatureGen(1)
    feature_generator.gen_node_features(G)

    name = basis_type + "_" + str(width_basis) + "_" + str(nb_shapes) + "_cycle"
    return G, role_id, name


def bagrid(nb_shapes=100, width_basis=60, feature_generator=None, m=5):
    """ Synthetic Graph #3:
    Start with Barabasi-Albert graph and attach grid-shaped subgraphs.
    Args:
        nb_shapes         :  The number of shapes (here 'grid') that should be added to the base graph.
        width_basis       :  The width of the basis graph (here 'Barabasi-Albert' random graph).
        feature_generator :  A `FeatureGenerator` for node features. If `None`, add constant features to nodes.
        m                 :  number of edges to attach to existing node (for BA graph)
    Returns:
        G                 :  A networkx graph
        role_id           :  Role ID for each node in synthetic graph.
        name              :  A graph identifier
    """
    basis_type = "ba"
    list_shapes = [["grid", 3]] * nb_shapes

    plt.figure(figsize=(8, 6), dpi=300)

    G, role_id, _ = build_graph(
        width_basis, basis_type, list_shapes, start=0, m=5
    )
    G = perturb([G], 0.01)[0]

    if feature_generator is None:
        feature_generator = ConstFeatureGen(1)
    feature_generator.gen_node_features(G)

    name = basis_type + "_" + str(width_basis) + "_" + str(nb_shapes)  + "_grid"
    return G, role_id, name

def treecycle(nb_shapes=100, width_basis=4, feature_generator=None, m=4):
    """ Synthetic Graph #4:
    Start with a tree and attach cycle-shaped subgraphs.
    Args:
        nb_shapes         :  The number of shapes (here 'houses') that should be added to the base graph.
        width_basis       :  The width of the basis graph (here a random 'Tree').
        feature_generator :  A `FeatureGenerator` for node features. If `None`, add constant features to nodes.
        m                 :  The tree depth.
    Returns:
        G                 :  A networkx graph
        role_id           :  Role ID for each node in synthetic graph
        name              :  A graph identifier
    """
    basis_type = "tree"
    list_shapes = [["cycle", 6]] * nb_shapes

    fig = plt.figure(figsize=(8, 6), dpi=300)

    G, role_id, plugins = build_graph(
        width_basis, basis_type, list_shapes, start=0
    )
    G = perturb([G], 0.01)[0]

    if feature_generator is None:
        feature_generator = ConstFeatureGen(1)
    feature_generator.gen_node_features(G)

    name = basis_type + "_" + str(width_basis) + "_" + str(nb_shapes)  + "_cycle"


    return G, role_id, name


def treegrid(nb_shapes=100, width_basis=4, feature_generator=None, m=4):
    """ Synthetic Graph #5:
    Start with a tree and attach grid-shaped subgraphs.
    Args:
        nb_shapes         :  The number of shapes (here 'houses') that should be added to the base graph.
        width_basis       :  The width of the basis graph (here a random 'grid').
        feature_generator :  A `FeatureGenerator` for node features. If `None`, add constant features to nodes.
        m                 :  The tree depth.
    Returns:
        G                 :  A networkx graph
        role_id           :  Role ID for each node in synthetic graph
        name              :  A graph identifier
    """
    basis_type = "tree"
    list_shapes = [["grid", 6]] * nb_shapes


    G, role_id, _ = build_graph(
        width_basis, basis_type, list_shapes, start=0
    )
    G = perturb([G], 0.1)[0]

    if feature_generator is None:
        feature_generator = ConstFeatureGen(1)
    feature_generator.gen_node_features(G)

    name = basis_type + "_" + str(width_basis) + "_" + str(nb_shapes)  + "_grid"


    return G, role_id, name
def treehouse(nb_shapes=100, width_basis=4, feature_generator=None, m=4):
    """ Synthetic Graph #5:
    Start with a tree and attach grid-shaped subgraphs.
    Args:
        nb_shapes         :  The number of shapes (here 'houses') that should be added to the base graph.
        width_basis       :  The width of the basis graph (here a random 'grid').
        feature_generator :  A `FeatureGenerator` for node features. If `None`, add constant features to nodes.
        m                 :  The tree depth.
    Returns:
        G                 :  A networkx graph
        role_id           :  Role ID for each node in synthetic graph
        name              :  A graph identifier
    """
    basis_type = "tree"
    list_shapes = [["house"]] * nb_shapes


    G, role_id, _ = build_graph(
        width_basis, basis_type, list_shapes, start=0
    )
    G = perturb([G], 0.1)[0]

    if feature_generator is None:
        feature_generator = ConstFeatureGen(1)
    feature_generator.gen_node_features(G)

    name = basis_type + "_" + str(width_basis) + "_" + str(nb_shapes)  + "_house"


    return G, role_id, name
def treefan(nb_shapes=100, width_basis=4, feature_generator=None, m=4):
    """ Synthetic Graph #5:
    Start with a tree and attach grid-shaped subgraphs.
    Args:
        nb_shapes         :  The number of shapes (here 'houses') that should be added to the base graph.
        width_basis       :  The width of the basis graph (here a random 'grid').
        feature_generator :  A `FeatureGenerator` for node features. If `None`, add constant features to nodes.
        m                 :  The tree depth.
    Returns:
        G                 :  A networkx graph
        role_id           :  Role ID for each node in synthetic graph
        name              :  A graph identifier
    """
    basis_type = "tree"
    list_shapes = [["fan"]] * nb_shapes


    G, role_id, _ = build_graph(
        width_basis, basis_type, list_shapes, start=0
    )
    G = perturb([G], 0.1)[0]

    if feature_generator is None:
        feature_generator = ConstFeatureGen(1)
    feature_generator.gen_node_features(G)

    name = basis_type + "_" + str(width_basis) + "_" + str(nb_shapes)  + "_fan"


    return G, role_id, name
def treeclique(nb_shapes=100, width_basis=4, feature_generator=None, m=4):
    """ Synthetic Graph #5:
    Start with a tree and attach grid-shaped subgraphs.
    Args:
        nb_shapes         :  The number of shapes (here 'houses') that should be added to the base graph.
        width_basis       :  The width of the basis graph (here a random 'grid').
        feature_generator :  A `FeatureGenerator` for node features. If `None`, add constant features to nodes.
        m                 :  The tree depth.
    Returns:
        G                 :  A networkx graph
        role_id           :  Role ID for each node in synthetic graph
        name              :  A graph identifier
    """
    basis_type = "tree"
    list_shapes = [["clique"]] * nb_shapes


    G, role_id, _ = build_graph(
        width_basis, basis_type, list_shapes, start=0
    )
    G = perturb([G], 0.1)[0]

    if feature_generator is None:
        feature_generator = ConstFeatureGen(1)
    feature_generator.gen_node_features(G)

    name = basis_type + "_" + str(width_basis) + "_" + str(nb_shapes)  + "_clique"


    return G, role_id, name

def treediamond(nb_shapes=100, width_basis=4, feature_generator=None, m=4):
    """ Synthetic Graph #5:
    Start with a tree and attach grid-shaped subgraphs.
    Args:
        nb_shapes         :  The number of shapes (here 'houses') that should be added to the base graph.
        width_basis       :  The width of the basis graph (here a random 'grid').
        feature_generator :  A `FeatureGenerator` for node features. If `None`, add constant features to nodes.
        m                 :  The tree depth.
    Returns:
        G                 :  A networkx graph
        role_id           :  Role ID for each node in synthetic graph
        name              :  A graph identifier
    """
    basis_type = "tree"
    list_shapes = [["diamond"]] * nb_shapes


    G, role_id, _ = build_graph(
        width_basis, basis_type, list_shapes, start=0
    )
    G = perturb([G], 0.1)[0]

    if feature_generator is None:
        feature_generator = ConstFeatureGen(1)
    feature_generator.gen_node_features(G)

    name = basis_type + "_" + str(width_basis) + "_" + str(nb_shapes)  + "_diamond"


    return G, role_id, name

def treestar(nb_shapes=100, width_basis=4, feature_generator=None, m=4):
    """ Synthetic Graph #5:
    Start with a tree and attach grid-shaped subgraphs.
    Args:
        nb_shapes         :  The number of shapes (here 'houses') that should be added to the base graph.
        width_basis       :  The width of the basis graph (here a random 'grid').
        feature_generator :  A `FeatureGenerator` for node features. If `None`, add constant features to nodes.
        m                 :  The tree depth.
    Returns:
        G                 :  A networkx graph
        role_id           :  Role ID for each node in synthetic graph
        name              :  A graph identifier
    """
    basis_type = "tree"
    list_shapes = [["star"]] * nb_shapes


    G, role_id, _ = build_graph(
        width_basis, basis_type, list_shapes, start=0
    )
    G = perturb([G], 0.1)[0]

    if feature_generator is None:
        feature_generator = ConstFeatureGen(1)
    feature_generator.gen_node_features(G)

    name = basis_type + "_" + str(width_basis) + "_" + str(nb_shapes)  + "_star"


    return G, role_id, name



    
def molloy_reed(g):
      all_degree =   np.array(g.degree())[:,1]
      degs = np.delete(all_degree,-1)
      k = degs.mean()
      k2 = np.mean(degs ** 2)
      beta = k2/k
      return beta 

def get_power_law_exponent(self):
    alpha = 1 + self.N / sum(np.log(self.degs/mink))
    return alpha

    
def global_feature(g): 
    subGraph = g#g.subgraph(np.arange(len(g)-1)) #for supernode
    M = len(subGraph.edges())
    N = len(subGraph)
    degs =   np.array(subGraph.degree())[:,1]
    min_k = np.min(degs)
    
    k1 = degs.mean()
    k2 = np.mean(degs** 2)
    div = k2 - k1**2
    A = nx.to_scipy_sparse_matrix(g, weight='weight',dtype=np.float,format='csr')
    adj_lams = np.sort(eigsh(A, k=N-1, which='LA', return_eigenvectors=False))
    nodes = N
    edges = M
    heterogeneity = div/k1
    density = (2*M)/(N*(N-1))
    resilience = molloy_reed(g)
    power_law_exponent = 1 + N / sum(np.log(degs/min_k))
    modularity = nx_comm.modularity(g,nx_comm.label_propagation_communities(g))
    varepsilons = nx.algorithms.distance_measures.eccentricity(g).values()
    eccentricity=  np.mean(list(varepsilons))
    diameter = nx.algorithms.distance_measures.diameter(g)
    radius = nx.algorithms.distance_measures.radius(g)
    spectral_radius=adj_lams[-1]
    spectral_gap=  adj_lams[-1]-adj_lams[-2]
    natural_connectivity= np.log2(sum(np.exp(np.real(adj_lams)))/len(adj_lams))
    global_efficiency = nx.algorithms.efficiency_measures.global_efficiency(g)
    global_properties =np.hstack((nodes, edges,density,resilience,heterogeneity,power_law_exponent, modularity,eccentricity,diameter,radius,spectral_radius,spectral_gap,natural_connectivity))
    return global_properties
    global_properties =np.hstack((nodes, edges,density,resilience,heterogeneity,power_law_exponent, modularity))
    return global_properties

In [10]:
%matplotlib inline
features = []
names = []
for function in [bahouse,bafan,baclique,badiamond,bacycle,bastar,bagrid]:
    for i in range(1,6):
        G_conf,_ ,name = function(nb_shapes=20*i,width_basis=300)
        feat = global_feature(G_conf)
        #plt.figure(figsize=(8, 6), dpi=300)
        #nx.draw(G_conf,nodelist=G_conf.nodes, font_color='white' ,node_size = 50, with_labels=False)
        features.append(feat)
        name = name+'_'+str(i)
        file_name =  "../Dataset/Validation/Motifs_Attached/"+name
        nx.write_edgelist(G_conf,file_name+str(".txt"))
        #plt.show()
        names.append(name)
print(names)
print(np.array(features))
    

feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32


/tmp/ipykernel_127857/3730912886.py:971: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  A = nx.to_scipy_sparse_matrix(g, weight='weight',dtype=np.float,format='csr')


feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
feat_dict[0]["

/tmp/ipykernel_127857/3730912886.py:684: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=(8, 6), dpi=300)


feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
['ba_300_20_house_1', 'ba_300_40_house_2', 'ba_300_60_house_3', 'ba_300_80_house_4', 'ba_300_100_house_5', 'ba_300_20_fan_1', 'ba_

...

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

In [ ]:
['ba_300_20_house_1', 'ba_300_40_house_2', 'ba_300_60_house_3', 'ba_300_80_house_4', 'ba_300_100_house_5', 'ba_300_20_fan_1', 'ba_300_40_fan_2', 'ba_300_60_fan_3', 'ba_300_80_fan_4', 'ba_300_100_fan_5', 'ba_300_20_clique_1', 'ba_300_40_clique_2', 'ba_300_60_clique_3', 'ba_300_80_clique_4', 'ba_300_100_clique_5', 'ba_300_20_diamond_1', 'ba_300_40_diamond_2', 'ba_300_60_diamond_3', 'ba_300_80_diamond_4', 'ba_300_100_diamond_5', 'ba_300_20_cycle_1', 'ba_300_40_cycle_2', 'ba_300_60_cycle_3', 'ba_300_80_cycle_4', 'ba_300_100_cycle_5', 'ba_300_20_star_1', 'ba_300_40_star_2', 'ba_300_60_star_3', 'ba_300_80_star_4', 'ba_300_100_star_5', 'ba_300_20_grid_1', 'ba_300_40_grid_2', 'ba_300_60_grid_3', 'ba_300_80_grid_4', 'ba_300_100_grid_5']
[[4.00000000e+02 1.63100000e+03 2.04385965e-02 1.68766871e+01 8.71256591e+00 1.89257056e+00 1.15867799e-01 6.66750000e+00 1.00000000e+01 5.00000000e+00 1.69257043e+01 9.07715843e+00 1.57792467e+01]
 [5.00000000e+02 1.77200000e+03 1.42044088e-02 1.51823828e+01 8.08694357e+00 2.04436815e+00 2.08628133e-01 7.08200000e+00 1.00000000e+01 6.00000000e+00 1.68127960e+01 9.48061445e+00 1.52936640e+01]
 [6.00000000e+02 1.91300000e+03 1.06455203e-02 1.45172594e+01 8.13404949e+00 2.19469597e+00 2.74691364e-01 7.42500000e+00 1.00000000e+01 6.00000000e+00 1.70888186e+01 9.31027559e+00 1.54282076e+01]
 [7.00000000e+02 2.05500000e+03 8.39975475e-03 1.33271665e+01 7.45022593e+00 2.32776871e+00 3.25087941e-01 7.44285714e+00 1.00000000e+01 5.00000000e+00 1.65173521e+01 8.69846801e+00 1.43816441e+01]
 [8.00000000e+02 2.19600000e+03 6.87108886e-03 1.21799544e+01 6.68531876e+00 2.42989583e+00 3.67429832e-01 7.88375000e+00 1.00000000e+01 6.00000000e+00 1.63827030e+01 9.07849329e+00 1.39945147e+01]
 [3.20000000e+02 1.50900000e+03 2.95650470e-02 1.65505469e+01 7.11414430e+00 1.50145460e+00 0.00000000e+00 4.35000000e+00 6.00000000e+00 3.00000000e+00 1.64147444e+01 9.01327024e+00 1.53652364e+01]
 [3.40000000e+02 1.53000000e+03 2.65486726e-02 1.71098398e+01 8.10457516e+00 1.53268247e+00 0.00000000e+00 4.51176471e+00 6.00000000e+00 4.00000000e+00 1.67086257e+01 8.82774475e+00 1.57013009e+01]
 [3.60000000e+02 1.55000000e+03 2.39863819e-02 1.56511778e+01 7.03534050e+00 1.55677128e+00 1.11022302e-16 4.83888889e+00 6.00000000e+00 4.00000000e+00 1.59649235e+01 8.37227247e+00 1.45465450e+01]
 [3.80000000e+02 1.57000000e+03 2.18025274e-02 1.73542529e+01 9.08588669e+00 1.58720073e+00 3.80279119e-03 4.57894737e+00 6.00000000e+00 3.00000000e+00 1.72521455e+01 9.67105289e+00 1.63240367e+01]
 [4.00000000e+02 1.59000000e+03 1.99248120e-02 1.59738912e+01 8.01918239e+00 1.61457243e+00 1.25469720e-03 4.83500000e+00 6.00000000e+00 4.00000000e+00 1.61305661e+01 8.61157732e+00 1.46329416e+01]
 [3.01000000e+02 1.50900000e+03 3.34219269e-02 1.80753836e+01 8.06155977e+00 3.01103627e+00 0.00000000e+00 3.67774086e+00 4.00000000e+00 3.00000000e+00 1.71853675e+01 9.60162570e+00 1.65650251e+01]
 [3.01000000e+02 1.53000000e+03 3.38870432e-02 1.68973510e+01 7.03323345e+00 2.90235903e+00 0.00000000e+00 3.73754153e+00 4.00000000e+00 3.00000000e+00 1.70450236e+01 9.65734147e+00 1.63625693e+01]
 [3.01000000e+02 1.55000000e+03 3.43300111e-02 1.81138158e+01 8.62551281e+00 2.91036712e+00 1.11022302e-16 3.65116279e+00 4.00000000e+00 3.00000000e+00 1.76710816e+01 9.47006322e+00 1.72656096e+01]
 [3.01000000e+02 1.57000000e+03 3.47729790e-02 1.72124877e+01 8.42606810e+00 2.87024677e+00 0.00000000e+00 3.62126246e+00 4.00000000e+00 3.00000000e+00 1.77873459e+01 9.24491130e+00 1.74333437e+01]
 [3.01000000e+02 1.59000000e+03 3.52159468e-02 1.73246753e+01 9.35974425e+00 2.81350785e+00 0.00000000e+00 3.53820598e+00 4.00000000e+00 3.00000000e+00 1.84072811e+01 9.71809447e+00 1.83275079e+01]
 [4.20000000e+02 1.75200000e+03 1.99113536e-02 1.51354286e+01 6.77985975e+00 2.21127964e+00 2.52124285e-01 6.62380952e+00 9.00000000e+00 5.00000000e+00 1.67481574e+01 9.15295321e+00 1.54526017e+01]
 [5.40000000e+02 2.01400000e+03 1.38390710e-02 1.36025354e+01 6.13259774e+00 3.30092385e+00 4.72365734e-01 7.21666667e+00 1.00000000e+01 6.00000000e+00 1.63947716e+01 8.58711238e+00 1.45800700e+01]
 [6.60000000e+02 2.27700000e+03 1.04704097e-02 1.29828571e+01 6.07496706e+00 3.74403754e+00 5.21477207e-01 7.46818182e+00 1.00000000e+01 6.00000000e+00 1.68873942e+01 9.14159621e+00 1.50003036e+01]
 [7.80000000e+02 2.54000000e+03 8.36048846e-03 1.23778566e+01 5.85843933e+00 4.13291798e+00 5.97531930e-01 7.51666667e+00 1.00000000e+01 6.00000000e+00 1.71466297e+01 9.45916821e+00 1.51327173e+01]
 [9.00000000e+02 2.80200000e+03 6.92621431e-03 1.11289286e+01 4.89717345e+00 4.46160252e+00 6.54201891e-01 7.91111111e+00 1.00000000e+01 6.00000000e+00 1.63216663e+01 8.34595288e+00 1.37372456e+01]
 [3.40000000e+02 1.53700000e+03 2.66701371e-02 1.70709635e+01 8.01998163e+00 1.52150584e+00 1.79874524e-02 5.23235294e+00 7.00000000e+00 4.00000000e+00 1.66816411e+01 8.75733363e+00 1.56624465e+01]
 [3.79000000e+02 1.58600000e+03 2.21412517e-02 1.62605678e+01 7.88218315e+00 1.55903868e+00 4.03431507e-02 5.49868074e+00 8.00000000e+00 5.00000000e+00 1.65588092e+01 9.07485610e+00 1.53281120e+01]
 [4.19000000e+02 1.64300000e+03 1.87619189e-02 1.69372525e+01 9.08204620e+00 1.59348607e+00 5.02945231e-02 5.66825776e+00 8.00000000e+00 5.00000000e+00 1.70018394e+01 9.10292887e+00 1.58219039e+01]
 [4.60000000e+02 1.68300000e+03 1.59420290e-02 1.56240713e+01 8.30233537e+00 1.63352164e+00 7.32776014e-02 5.76739130e+00 8.00000000e+00 4.00000000e+00 1.65938943e+01 9.13459723e+00 1.50985585e+01]
 [5.00000000e+02 1.74100000e+03 1.39559118e-02 1.67851192e+01 9.81658587e+00 1.66428793e+00 8.64211481e-02 5.88400000e+00 8.00000000e+00 5.00000000e+00 1.75279810e+01 1.01066201e+01 1.63250652e+01]
 [3.20000000e+02 1.50900000e+03 2.95650470e-02 1.58863109e+01 6.45012840e+00 1.50062671e+00 0.00000000e+00 4.50625000e+00 6.00000000e+00 4.00000000e+00 1.62201291e+01 8.77793075e+00 1.50846927e+01]
 [3.40000000e+02 1.53000000e+03 2.65486726e-02 1.73157895e+01 8.31045752e+00 1.53360766e+00 0.00000000e+00 4.29705882e+00 6.00000000e+00 3.00000000e+00 1.66841637e+01 8.87666187e+00 1.56660613e+01]
 [3.60000000e+02 1.55000000e+03 2.39863819e-02 1.65792191e+01 7.96308244e+00 1.56056692e+00 1.11022302e-16 4.56388889e+00 6.00000000e+00 4.00000000e+00 1.65848006e+01 8.88756965e+00 1.54400607e+01]
 [3.80000000e+02 1.57000000e+03 2.18025274e-02 1.67113730e+01 8.44321153e+00 1.58785983e+00 0.00000000e+00 4.63157895e+00 6.00000000e+00 4.00000000e+00 1.66933575e+01 8.63018434e+00 1.55183915e+01]
 [4.00000000e+02 1.59000000e+03 1.99248120e-02 1.59707455e+01 8.01603774e+00 1.61380447e+00 0.00000000e+00 4.87000000e+00 6.00000000e+00 4.00000000e+00 1.64697727e+01 9.06985039e+00 1.51217662e+01]
 [4.80000000e+02 1.75200000e+03 1.52400835e-02 1.46550543e+01 7.34783105e+00 1.99046629e+00 2.39560528e-01 9.10000000e+00 1.30000000e+01 7.00000000e+00 1.64382773e+01 8.96145889e+00 1.48127864e+01]
 [6.60000000e+02 2.01400000e+03 9.26104750e-03 1.30968703e+01 6.98833017e+00 2.21285509e+00 3.89301084e-01 9.91969697e+00 1.40000000e+01 8.00000000e+00 1.61598788e+01 8.68185077e+00 1.39513859e+01]
 [8.40000000e+02 2.27700000e+03 6.46177422e-03 1.21942004e+01 6.76829475e+00 2.39553424e+00 5.14091479e-01 1.03047619e+01 1.40000000e+01 8.00000000e+00 1.64142427e+01 8.96903305e+00 1.39696439e+01]
 [1.02000000e+03 2.54000000e+03 4.88752910e-03 1.17755022e+01 6.79126139e+00 2.54651119e+00 5.79808575e-01 1.02647059e+01 1.40000000e+01 7.00000000e+00 1.66424206e+01 8.59273179e+00 1.40183829e+01]
 [1.20000000e+03 2.80200000e+03 3.89491243e-03 1.09593002e+01 6.28610278e+00 2.66443972e+00 6.29089436e-01 1.07666667e+01 1.40000000e+01 8.00000000e+00 1.67332203e+01 8.58258582e+00
  1.39146225e+01]]

In [13]:
features = []
names = []
for function in [treehouse,treefan,treeclique,treediamond,treecycle,treestar,treegrid]:
    for i in range(1,6):
        G_conf,_ ,name = function(nb_shapes=20*i,width_basis=8)
        print(name)
        feat = global_feature(G_conf)
        #plt.figure(figsize=(8, 6), dpi=300)
        #nx.draw(G_conf,nodelist=G_conf.nodes, font_color='white' ,node_size = 50, with_labels=False)
        features.append(feat)
        name = name+'_'+str(i)
        file_name =  "../Dataset/Validation/Motifs_Attached/"+name
        nx.write_edgelist(G_conf,file_name+str(".txt"))
        plt.show()
        names.append(name)
print(names)
print(np.array(features))
    

feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
tree_8_20_house


/tmp/ipykernel_127857/3730912886.py:971: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  A = nx.to_scipy_sparse_matrix(g, weight='weight',dtype=np.float,format='csr')


feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
tree_8_40_house
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
tree_8_60_house
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
tree_8_80_house
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
tree_8_100_house
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
tree_8_20_fan
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
tree_8_40_fan
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
tree_8_60_fan
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
tree_8_80_fan
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
tree_8_100_fan
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
tree_8_20_clique
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
tree_8_40_clique
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
tree_8_60_clique
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
tree_8_80_clique
feat_dict[0]["feat"]: float32
G.nodes[0]["

<Figure size 2400x1800 with 0 Axes>

feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
tree_8_40_cycle


...

feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
tree_8_80_cycle


<Figure size 2400x1800 with 0 Axes>

feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
tree_8_100_cycle


<Figure size 2400x1800 with 0 Axes>

feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
tree_8_20_star
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
tree_8_40_star
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
tree_8_60_star
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
tree_8_80_star
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
tree_8_100_star
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
tree_8_20_grid
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
tree_8_40_grid
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
tree_8_60_grid
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
tree_8_80_grid
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32
tree_8_100_grid
['tree_8_20_house_1', 'tree_8_40_house_2', 'tree_8_60_house_3', 'tree_8_80_house_4', 'tree_8_100_house_5', 'tree_8_20_fan_1', 'tree_8_40_fan_2', 'tree_8_60_fan_3', 'tree_8_80_fan_4', 'tree_8_100_fan_5', 'tree_8_20_clique_1', 'tree_8_40_clique_2', 'tree_8_60_clique_3',

In [ ]:
['tree_8_20_house_1', 'tree_8_40_house_2', 'tree_8_60_house_3', 'tree_8_80_house_4', 'tree_8_100_house_5', 'tree_8_20_fan_1', 'tree_8_40_fan_2', 'tree_8_60_fan_3', 'tree_8_80_fan_4', 'tree_8_100_fan_5', 'tree_8_20_clique_1', 'tree_8_40_clique_2', 'tree_8_60_clique_3', 'tree_8_80_clique_4', 'tree_8_100_clique_5', 'tree_8_20_diamond_1', 'tree_8_40_diamond_2', 'tree_8_60_diamond_3', 'tree_8_80_diamond_4', 'tree_8_100_diamond_5', 'tree_8_20_cycle_1', 'tree_8_40_cycle_2', 'tree_8_60_cycle_3', 'tree_8_80_cycle_4', 'tree_8_100_cycle_5', 'tree_8_20_star_1', 'tree_8_40_star_2', 'tree_8_60_star_3', 'tree_8_80_star_4', 'tree_8_100_star_5', 'tree_8_20_grid_1', 'tree_8_40_grid_2', 'tree_8_60_grid_3', 'tree_8_80_grid_4', 'tree_8_100_grid_5']
[[6.11000000e+02 7.15000000e+02 3.83676317e-03 2.86554622e+00 5.23910132e-01 2.39334009e+00 6.08732945e-01 1.58248773e+01 1.90000000e+01 1.10000000e+01 3.13717291e+00 1.96699064e-02 1.49381166e+00]
 [7.11000000e+02 8.69000000e+02 3.44287950e-03 2.95619597e+00 5.11827132e-01 2.29583498e+00 5.90675366e-01 1.67651195e+01 2.10000000e+01 1.10000000e+01 3.32063587e+00 1.40361693e-01 1.57129500e+00]
 [8.11000000e+02 1.02300000e+03 3.11458191e-03 3.01761252e+00 4.93806447e-01 2.23288931e+00 5.88629651e-01 1.66683107e+01 2.10000000e+01 1.10000000e+01 3.32968703e+00 1.44002672e-01 1.62799570e+00]
 [9.11000000e+02 1.17700000e+03 2.83953149e-03 3.03659574e+00 4.54259140e-01 2.18280004e+00 5.84434456e-01 1.72425906e+01 2.20000000e+01 1.10000000e+01 3.36282660e+00 5.87622370e-02 1.66797150e+00]
 [1.01100000e+03 1.33100000e+03 2.60696693e-03 3.07969925e+00 4.45851457e-01 2.14949380e+00 5.74794207e-01 1.71819980e+01 2.20000000e+01 1.10000000e+01 3.33636289e+00 7.48231412e-02 1.70107888e+00]
 [5.31000000e+02 5.83000000e+02 4.14312618e-03 2.76137339e+00 5.64005905e-01 2.56467559e+00 6.22267858e-01 1.48399247e+01 1.80000000e+01 9.00000000e+00 3.03925102e+00 9.81006438e-02 1.37922140e+00]
 [5.51000000e+02 6.05000000e+02 3.99274047e-03 2.80479735e+00 6.07298526e-01 2.58162387e+00 6.28787651e-01 1.42323049e+01 1.70000000e+01 9.00000000e+00 3.14082606e+00 8.53085315e-02 1.38653185e+00]
 [5.71000000e+02 6.27000000e+02 3.85288967e-03 2.79569034e+00 5.98111263e-01 2.57092316e+00 6.39845954e-01 1.50945709e+01 1.80000000e+01 9.00000000e+00 3.16213833e+00 1.79184507e-01 1.38515906e+00]
 [5.91000000e+02 6.49000000e+02 3.72250423e-03 2.81804163e+00 6.20363490e-01 2.58064900e+00 6.15919715e-01 1.52961083e+01 1.80000000e+01 9.00000000e+00 3.02479204e+00 9.30460120e-02 1.38770733e+00]
 [6.11000000e+02 6.71000000e+02 3.60065466e-03 2.83296048e+00 6.35195290e-01 2.58679889e+00 6.20928569e-01 1.50654664e+01 1.80000000e+01 9.00000000e+00 3.07985555e+00 9.70343588e-02 1.39028725e+00]
 [5.12000000e+02 5.83000000e+02 4.45664139e-03 2.79057592e+00 8.08419543e-01 2.49354091e+00 5.89808732e-01 1.32988281e+01 1.60000000e+01 8.00000000e+00 4.75750785e+00 1.71425015e+00 1.49312387e+00]
 [5.12000000e+02 6.05000000e+02 4.62481654e-03 2.87692308e+00 1.74085098e+00 2.45817485e+00 5.87253603e-01 1.22910156e+01 1.60000000e+01 8.00000000e+00 6.53569657e+00 3.44113760e+00 1.98354161e+00]
 [5.12000000e+02 6.27000000e+02 4.79299168e-03 2.92797320e+00 3.20947344e+00 2.41222029e+00 5.56698946e-01 1.01601562e+01 1.40000000e+01 7.00000000e+00 7.92236453e+00 4.84820804e+00 3.00074704e+00]
 [5.12000000e+02 6.49000000e+02 4.96116683e-03 2.95402299e+00 5.16746316e+00 2.36392366e+00 5.35753714e-01 9.80664062e+00 1.30000000e+01 7.00000000e+00 9.11905104e+00 6.14040853e+00 4.35548764e+00]
 [5.12000000e+02 6.71000000e+02 5.12934198e-03 3.00000000e+00 7.60692413e+00 2.32226955e+00 5.02950642e-01 7.45117188e+00 1.00000000e+01 5.00000000e+00 1.01875396e+01 7.05113944e+00 5.77118447e+00]
 [6.31000000e+02 8.47000000e+02 4.26131361e-03 3.35147929e+00 6.68383050e-01 2.20098686e+00 6.88471290e-01 1.56798732e+01 2.00000000e+01 1.10000000e+01 4.35369129e+00 5.29604404e-02 2.07954711e+00]
 [7.51000000e+02 1.13300000e+03 4.02308034e-03 3.68435013e+00 6.67597073e-01 2.04599054e+00 7.28109379e-01 1.57283622e+01 2.00000000e+01 1.10000000e+01 4.43326177e+00 5.98464656e-02 2.41861190e+00]
 [8.71000000e+02 1.41900000e+03 3.74519973e-03 3.86410166e+00 6.07779124e-01 1.95186782e+00 7.47729520e-01 1.61814007e+01 2.00000000e+01 1.10000000e+01 4.52163055e+00 1.20515826e-01 2.62655247e+00]
 [9.91000000e+02 1.70500000e+03 3.47572598e-03 4.03229595e+00 5.91289346e-01 1.90083610e+00 7.66346351e-01 1.48990918e+01 1.90000000e+01 1.10000000e+01 4.58019205e+00 2.09155890e-02 2.77043627e+00]
 [1.11100000e+03 1.99100000e+03 3.22897155e-03 4.10507793e+00 5.20813960e-01 1.85553657e+00 7.77800327e-01 1.51584158e+01 1.90000000e+01 1.10000000e+01 4.57599163e+00 3.95854930e-02 2.86805134e+00]
 [6.31000000e+02 6.66000000e+02 3.35069051e-03 2.54736842e+00 4.35611523e-01 2.59889825e+00 6.45854188e-01 1.92266244e+01 2.40000000e+01 1.20000000e+01 2.81830971e+00 7.62507451e-02 1.31321256e+00]
 [7.51000000e+02 8.17000000e+02 2.90102086e-03 2.56985294e+00 3.93389801e-01 2.48808356e+00 6.25336897e-01 1.95259654e+01 2.40000000e+01 1.20000000e+01 2.90294576e+00 6.02127736e-02 1.34299836e+00]
 [8.71000000e+02 9.69000000e+02 2.55750426e-03 2.58574380e+00 3.60110616e-01 2.41646207e+00 6.00575530e-01 1.98140069e+01 2.40000000e+01 1.20000000e+01 2.90010860e+00 5.04971390e-02 1.36522252e+00]
 [9.91000000e+02 1.11200000e+03 2.26686644e-03 2.56705671e+00 3.22348983e-01 2.37737050e+00 5.97939095e-01 1.99162462e+01 2.40000000e+01 1.20000000e+01 2.90950210e+00 3.59844916e-02 1.36974833e+00]
 [1.11100000e+03 1.26900000e+03 2.05804364e-03 2.58123028e+00 2.96343819e-01 2.32676138e+00 5.85375888e-01 2.00738074e+01 2.40000000e+01 1.20000000e+01 2.93749250e+00 4.89112638e-02 1.38767332e+00]
 [5.31000000e+02 5.83000000e+02 4.14312618e-03 2.76480687e+00 5.67436437e-01 2.56370465e+00 6.25223235e-01 1.49359699e+01 1.80000000e+01 9.00000000e+00 3.05620799e+00 1.35058180e-01 1.38014119e+00]
 [5.51000000e+02 6.05000000e+02 3.99274047e-03 2.80148883e+00 6.03992740e-01 2.57715209e+00 6.23942354e-01 1.51343013e+01 1.80000000e+01 9.00000000e+00 3.10805763e+00 1.35779157e-01 1.38697963e+00]
 [5.71000000e+02 6.27000000e+02 3.85288967e-03 2.80526736e+00 6.07680641e-01 2.57548879e+00 6.30441916e-01 1.53012259e+01 1.80000000e+01 9.00000000e+00 3.02101061e+00 9.77187240e-02 1.38609480e+00]
 [5.91000000e+02 6.49000000e+02 3.72250423e-03 2.80724749e+00 6.09577666e-01 2.57690433e+00 6.29260139e-01 1.55465313e+01 1.80000000e+01 9.00000000e+00 3.17214071e+00 7.40320354e-02 1.39089755e+00]
 [6.11000000e+02 6.71000000e+02 3.60065466e-03 2.82252051e+00 6.24763099e-01 2.58173187e+00 6.10267612e-01 1.54173486e+01 1.80000000e+01 9.00000000e+00 3.05295496e+00 1.02951782e-01 1.38839719e+00]
 [1.23100000e+03 1.90300000e+03 2.51365471e-03 3.52839117e+00 4.35792731e-01 1.96977031e+00 5.13715656e-01 1.49512591e+01 1.80000000e+01 1.20000000e+01 3.99096859e+00 3.75323476e-02 1.93406894e+00]
 [1.95100000e+03 3.24500000e+03 1.70589704e-03 3.68988903e+00 3.62869028e-01 1.88797249e+00 5.05793861e-01 1.54787289e+01 1.90000000e+01 1.20000000e+01 4.00903026e+00 7.93874351e-02 2.05247189e+00]
 [2.67100000e+03 4.58700000e+03 1.28639276e-03 3.74640209e+00 3.11352701e-01 1.85275206e+00 4.92261616e-01 1.61463871e+01 2.10000000e+01 1.30000000e+01 4.01596445e+00 7.75234578e-02 2.10318175e+00]
 [3.39100000e+03 5.92900000e+03 1.03153498e-03 3.79352227e+00 2.96316199e-01 1.83533189e+00 4.93715537e-01 1.68596284e+01 2.30000000e+01 1.30000000e+01 4.08792989e+00 7.97548799e-02 2.13446061e+00]
 [4.11100000e+03 7.27100000e+03 8.60666386e-04 3.80990371e+00 2.72315946e-01 1.82294486e+00 4.87519984e-01 1.62816833e+01 2.10000000e+01 1.40000000e+01 4.10287047e+00 9.62138460e-02 2.15217682e+00]]


## Real World 

In [4]:
import os

import networkx as nx
import networkx.algorithms.community as nx_comm
import numpy as np
import math
from scipy.sparse.linalg import eigsh

def molloy_reed(g):
  all_degree =   np.array(g.degree())[:,1]
  degs = np.delete(all_degree,-1)
  k = degs.mean()
  k2 = np.mean(degs ** 2)
  beta = k2/k
  return beta 

def get_power_law_exponent(self):
    alpha = 1 + self.N / sum(np.log(self.degs/mink))
    return alpha

    
def global_feature(g): 
    subGraph = g#g.subgraph(np.arange(len(g)-1)) #for supernode
    M = len(subGraph.edges())
    N = len(subGraph)
    degs =   np.array(subGraph.degree())[:,1]
    min_k = np.min(degs)
    
    k1 = degs.mean()
    k2 = np.mean(degs** 2)
    div = k2 - k1**2
    A = nx.to_scipy_sparse_matrix(g, weight='weight',dtype=np.float,format='csr')
    adj_lams = np.sort(eigsh(A, k=N-1, which='LA', return_eigenvectors=False))
    nodes = N
    edges = M
    heterogeneity = div/k1
    density = (2*M)/(N*(N-1))
    resilience = molloy_reed(g)
    power_law_exponent = 1 + N / sum(np.log(degs/min_k))
    modularity = nx_comm.modularity(g,nx_comm.label_propagation_communities(g))
    try:
        varepsilons = nx.algorithms.distance_measures.eccentricity(g).values()
        eccentricity=  np.mean(list(varepsilons))
        diameter = nx.algorithms.distance_measures.diameter(g)
        radius = nx.algorithms.distance_measures.radius(g)
    except:
        lcc = max(nx.connected_component(G), key=len)
        Subgraph = G.subgraph(lcc)
        varepsilons = nx.algorithms.distance_measures.eccentricity(Subgraph).values()
        eccentricity=  np.mean(list(varepsilons))
        diameter = nx.algorithms.distance_measures.diameter(Subgraph)
        radius = nx.algorithms.distance_measures.radius(Subgraph)
    spectral_radius=adj_lams[-1]
    spectral_gap=  adj_lams[-1]-adj_lams[-2]
    natural_connectivity= np.log2(sum(np.exp(np.real(adj_lams)))/len(adj_lams))
    global_efficiency = nx.algorithms.efficiency_measures.global_efficiency(g)
    assortativity = nx.algorithms.assortativity.degree_assortativity_coefficient(g)
    print(heterogeneity,diameter,assortativity)
    global_properties =np.hstack((nodes, edges,density,resilience,heterogeneity,power_law_exponent, modularity,eccentricity,diameter,radius,spectral_radius,spectral_gap,natural_connectivity))
    return global_properties
features = []
names = []
path_of_the_directory= "real"

filename =   ['heterogeneous_30','heterogeneous_50','heterogeneous_100','heterogeneous_500']
filepath= ["../Dataset/HomogeneityGraph/"+file+".txt" for file in filename]  

for file in filepath:
    print(file)
    fh = open(file, "rb")
    G_conf = nx.read_edgelist(fh)
    fh.close()
    lcc = max(nx.connected_components(G_conf), key=len)
    G_conf = G_conf.subgraph(lcc)
    nodes = G_conf.nodes()
    map = {n:int(i) for i, n in enumerate(nodes)}
    #GRAPH = reset(add_super_node(nx.relabel_nodes(GRAPH, map)))
    G_conf =  nx.relabel_nodes(G_conf, map)
    G_conf.remove_edges_from(nx.selfloop_edges(G_conf))
    G_conf.remove_nodes_from(nx.isolates(G_conf))
    feat = global_feature(G_conf)
    features.append(feat)
print(np.array(features))

../Dataset/HomogeneityGraph/homogeneous_30.txt
../Dataset/HomogeneityGraph/homogeneous_50.txt
../Dataset/HomogeneityGraph/homogeneous_100.txt
../Dataset/HomogeneityGraph/homogeneous_500.txt


/tmp/ipykernel_182974/4101885454.py:32: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  A = nx.to_scipy_sparse_matrix(g, weight='weight',dtype=np.float,format='csr')


[[2.90000000e+01 4.80000000e+01 1.18226601e-01 3.69473684e+00
  3.56321839e-01 1.88549419e+00 3.29861111e-01 4.86206897e+00
  6.00000000e+00 4.00000000e+00 3.62663654e+00 6.01718060e-01
  2.10565352e+00]
 [5.00000000e+01 7.80000000e+01 6.36734694e-02 3.64516129e+00
  5.08205128e-01 1.96414874e+00 4.16009204e-01 6.46000000e+00
  8.00000000e+00 5.00000000e+00 3.61924707e+00 3.16089730e-01
  1.98560318e+00]
 [9.40000000e+01 1.50000000e+02 3.43170899e-02 3.95973154e+00
  7.55177305e-01 1.97689616e+00 5.22444444e-01 8.12765957e+00
  1.10000000e+01 6.00000000e+00 4.13464925e+00 3.96779272e-01
  2.10133017e+00]
 [4.66000000e+02 7.87000000e+02 7.26383313e-03 4.17228226e+00
  7.92584433e-01 1.91577842e+00 5.77348882e-01 1.35364807e+01
  1.70000000e+01 1.00000000e+01 4.34311025e+00 1.69668607e-01
  2.19029031e+00]]


In [ ]:
 ['heterogeneous_30','heterogeneous_50','heterogeneous_100','heterogeneous_500']
[[2.30000000e+01 3.10000000e+01 1.22529644e-01 3.32786885e+00 5.94670407e-01 2.16148170e+00 3.22060354e-01 5.34782609e+00 7.00000000e+00 4.00000000e+00 3.39510706e+00 8.07211753e-01 1.82678582e+00]
 [3.60000000e+01 5.20000000e+01 8.25396825e-02 3.32000000e+00 4.57264957e-01 2.03397491e+00 4.56545858e-01 6.44444444e+00 8.00000000e+00 4.00000000e+00 3.32751135e+00 2.68852896e-01 1.86972708e+00]
 [7.60000000e+01 1.18000000e+02 4.14035088e-02 4.25957447e+00 1.14049955e+00 2.04041130e+00 5.17451882e-01 7.89473684e+00 1.00000000e+01 6.00000000e+00 4.43246189e+00 8.70449976e-01 2.16475129e+00]
 [4.14000000e+02 6.39000000e+02 7.47447100e-03 4.36726703e+00 1.27767572e+00 2.07480515e+00 5.88949625e-01 1.41908213e+01 1.90000000e+01 1.10000000e+01 4.68377251e+00 3.33456762e-01 2.13131275e+00]]
 ['homogeneous_30','homogeneous_50','homogeneous_100','homogeneous_500']
[[2.90000000e+01 4.80000000e+01 1.18226601e-01 3.69473684e+00 3.56321839e-01 1.88549419e+00 3.29861111e-01 4.86206897e+00 6.00000000e+00 4.00000000e+00 3.62663654e+00 6.01718060e-01 2.10565352e+00]
 [5.00000000e+01 7.80000000e+01 6.36734694e-02 3.64516129e+00 5.08205128e-01 1.96414874e+00 4.16009204e-01 6.46000000e+00 8.00000000e+00 5.00000000e+00 3.61924707e+00 3.16089730e-01 1.98560318e+00]
 [9.40000000e+01 1.50000000e+02 3.43170899e-02 3.95973154e+00 7.55177305e-01 1.97689616e+00 5.22444444e-01 8.12765957e+00 1.10000000e+01 6.00000000e+00 4.13464925e+00 3.96779272e-01 2.10133017e+00]
 [4.66000000e+02 7.87000000e+02 7.26383313e-03 4.17228226e+00 7.92584433e-01 1.91577842e+00 5.77348882e-01 1.35364807e+01 1.70000000e+01 1.00000000e+01 4.34311025e+00 1.69668607e-01 2.19029031e+00]]
    